In [1]:
import findspark

findspark.init()

from pyspark import SparkContext
sc=SparkContext("local","quiz")

21/11/03 02:55:24 WARN Utils: Your hostname, jeongsobin-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.30.1.53 instead (on interface en0)
21/11/03 02:55:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/11/03 02:55:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
data=sc.textFile("pagerank_data.txt")

In [3]:
pro_data=data.map(lambda x:[int(j) for j in x.split(" ")]).map(lambda x:(x[0],list(x[1:])))

# Teleport

## 기존방식

spider trap과 dead end를 피하기 위해 사용한 기존방식은 랜덤워커가 일정한 작은 확률로는 다른곳으로 텔레포트 할수있는데

이 확률이 어떤 페이지든지 동일한 확률을 가진다.


## 새로운방식

모든 페이지가 동일한 확률을 가져서 랜덤으로 가는거말고 bias를 주어서 주제에 따라 좀더 개인화된 텔레포트를 가능하게 할수있다

random walk마다 bias를 주면 텔레포트 할때 현재 페이지가 속해있는 집합에서 같은 집합내에 다른 페이지로 텔레포트 하기 때문에 이방식을 선택했다.

집합은 해당 주제와 관련된 페이지만을 포함하고 집합으로 텔레포트 발생마다 다른 벡터를 얻는다.

주어진 데이터는 숫자들로만 이루어져 있기 때문에 같은 집합인지 아닌지 판별하는 기준을 jaccard coefficient를 이용해 구별했다.

### pagerank에서 bias 주는방법

dv+1/N(1-d)에서 N을 전체페이지길이에서 도착할 노드가 속한 집합S의 길이로 바꿔주어서 teleport 집합에 있는 모든 페이지에 같은 가중치를 부여한다.

$$ \beta M_{ij}+1/|N|(1-\beta) $$
$$ \Downarrow $$
$$ \beta M_{ij}+1/|S|(1-\beta) $$  

outlink가 0인경우에는 기존방식을 사용

### 구현 방법

page id별로 가지고있는 outgoing link들 끼리의 유사도를 계산하기 위해 jaccard similarity를 이용했고

주어진 threshhold보다 높은 outgoing link들을 같은 집합으로 취급해 키별로 각 집합의 길이를 구했으며 이를 S의 길이로 사용했다.

In [4]:
dictionary=sc.broadcast(dict(pro_data.collect()))

In [5]:
def jaccard(x,y):
    if x is None:
        union=y
    elif y is None:
        union=x
    else:
        union=list(set(x) | set(y))
        
    intersection=list(set(x) & set(y))
    
    return len(intersection)/len(union)



In [6]:
jaccard_result=pro_data.flatMap(lambda x:[(x[0],y,jaccard(x[1],dictionary.value[y])) for y in x[1]])\
                .filter(lambda x:x[2]>=0.205).map(lambda x:(x[0],1)).reduceByKey(lambda x,y:x+y).collect()


/opt/homebrew/Cellar/apache-spark/3.1.2/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


In [51]:
set2=jaccard_result.copy()

set2.append((17094,1))

result=sc.parallelize(set2).sortByKey().map(lambda x:x[1]).collect()

set_len=sc.broadcast(result)

/opt/homebrew/Cellar/apache-spark/3.1.2/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


In [74]:
N=30000
d=0.85
convergenceTol=1e-25
compare_convergence=[]
maxIter=20
pr=sc.broadcast([1./float(N) for _ in range(N)])

for i in range(maxIter):
    data_result=pro_data.flatMap(lambda x:[(target,pr.value[x[0]]/len(x[1])) for target in x[1]])\
    .reduceByKey(lambda x,y:x+y).sortBy(lambda x: x[0]).map(lambda v:(v[0],d*v[1]+(1/set_len.value[v[0]])*(1-d)))
    
    
    
    pr_new=data_result.map(lambda t:t[1]).collect()
    
    pr_new.insert(0,0)
    
    pr=sc.broadcast(pr_new)
    
    
    compare_convergence.append(abs(err))
 

/opt/homebrew/Cellar/apache-spark/3.1.2/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


In [79]:
final_result=data_result.sortBy(lambda x:x[1],False).collect()

[(17094, 0.1506908251054157),
 (70, 0.11204408954812414),
 (41, 0.10967163110274632),
 (13, 0.10932762728048163),
 (42, 0.10925278765111598),
 (27, 0.10903679296790819),
 (5, 0.10902999086379064),
 (25, 0.10899730542842065),
 (46, 0.10897329491113601),
 (34, 0.10897264654300227),
 (49, 0.10895985234874594),
 (4, 0.10889537247765049),
 (7, 0.10888534841991192),
 (16, 0.10884295158554222),
 (38, 0.10881693024804547),
 (6, 0.10874098280367697),
 (66, 0.1086969518590809),
 (53, 0.10861876160396995),
 (54, 0.10858123630919751),
 (14, 0.10853874337192045),
 (40, 0.1085361659531389),
 (51, 0.10852976407251202),
 (43, 0.10847151945248999),
 (21, 0.10846445592498291),
 (12, 0.10845932596602258),
 (11, 0.10844700826795399),
 (44, 0.10844293622419728),
 (50, 0.10844267604771181),
 (67, 0.1084405270723222),
 (18, 0.10843764409527865),
 (48, 0.10843097221339015),
 (61, 0.1084178824598149),
 (55, 0.1084149664154365),
 (20, 0.10841288984400506),
 (9, 0.10840399011429315),
 (19, 0.10840302628983771),


In [ ]:
import pickle
with open('data_biased.pkl','wb') as f:
    pickle.dump(final_result,f)